In [10]:
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

RendererRegistry.enable('jupyterlab')

In [11]:
# Suppress warnings.
import warnings
warnings.filterwarnings("ignore")

# Suppress js errors.
alt.renderers.enable('jupyterlab')

In [12]:
DATA_DIR = 'csv/'
IMG_DIR = 'img/'

### Data: Monsoon

In [43]:
monsoon = pd.read_csv(DATA_DIR + 'Monsoon_data.csv', parse_dates=['Date'])
monsoon.set_index('Date', drop=False, inplace=True)

In [42]:
monsoon['dataset'] = 'monsoon'
monsoon['date_str'] = monsoon['Date'].astype(str)
monsoon['year'] = monsoon['date_str'].str[:4]
monsoon['month'] = pd.to_numeric(monsoon['date_str'].str[5:7])

# This could be more elegant.
monsoon.loc[monsoon['month'] <= 12, 'quarter'] = 4
monsoon.loc[monsoon['month'] <= 9, 'quarter'] = 3
monsoon.loc[monsoon['month'] <= 6, 'quarter'] = 2
monsoon.loc[monsoon['month'] <= 3, 'quarter'] = 1

In [41]:
# monsoon.head(20)

### Data: Olou

In [7]:
olou = pd.read_csv(DATA_DIR + 'Olou_counts.csv', parse_dates=['Date'])
olou.set_index('Date', drop=False, inplace=True)

In [28]:
olou['dataset'] = 'olou'
olou['date_str'] = olou['Date'].astype(str)
olou['year'] = olou['date_str'].str[:4]
olou['month'] = pd.to_numeric(olou['date_str'].str[5:7])

# This could be more elegant.
olou.loc[olou['month'] <= 12, 'quarter'] = 4
olou.loc[olou['month'] <= 9, 'quarter'] = 3
olou.loc[olou['month'] <= 6, 'quarter'] = 2
olou.loc[olou['month'] <= 3, 'quarter'] = 1

In [39]:
# olou.head()

### Data: Monsoon + Olou

In [32]:
dataframes = [monsoon, olou]
dataset = pd.concat(dataframes)

In [33]:
dataset.head()

,Date,Precip,date_str,year,month,quarter,dataset,Counts
Date,,,,,,,,
1964-01-01,1964-01-01,1.4,1964-01-01,1964,1,1.0,monsoon,NaN
1964-02-01,1964-02-01,5.7,1964-02-01,1964,2,1.0,monsoon,NaN
1964-03-01,1964-03-01,8.3,1964-03-01,1964,3,1.0,monsoon,NaN
1964-04-01,1964-04-01,24.8,1964-04-01,1964,4,2.0,monsoon,NaN
1964-05-01,1964-05-01,43.7,1964-05-01,1964,5,2.0,monsoon,NaN


In [436]:
display(dataset['Counts'].max())

6843.0

### Interactive Visualization

In [491]:
# Selections.
select = alt.selection_interval(
    bind="scales",
    encodings=['x', 'y']
)

# # Resize height.
# bind_range_height = alt.binding_range(min=100, max=1200, name='Height: ')
# param_height = alt.param('height', bind=bind_range_height)

# Resize width.
bind_range_width = alt.binding_range(min=400, max=1200, name='Width: ')
param_width = alt.param('width', bind=bind_range_width)

olou_domain = [dataset['Counts'].min(), dataset['Counts'].max()]
monsoon_domain = [dataset['Precip'].min(), dataset['Precip'].max()]

# Monsoon precipitation.
line_chart_monsoon = alt.Chart(dataset[np.isnan(dataset['Precip']) == False]).mark_line(
    interpolate='basis',
    strokeWidth=2.5,
    opacity=0.25
).encode(
    x=alt.X('Date:T', title=""),
    y=alt.Y('Precip:Q', scale=alt.Scale(domain=monsoon_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).add_params(
    # param_width,
    select
)

# Monsoon precipitation rolling average.
moving_avg_monsoon = alt.Chart(dataset[np.isnan(dataset['Precip']) == False]).transform_window(
    rolling_mean='mean(Precip)',
    frame=[-3, 3],
).mark_line(
    color='green',
    strokeWidth=2,
    interpolate='basis'
).encode(
    x=alt.X('Date:T'),
    y=alt.Y('rolling_mean:Q', scale=alt.Scale(domain=monsoon_domain))
)

# Olou cosmic ray counts.
line_chart_olou = alt.Chart(dataset[np.isnan(dataset['Counts']) == False]).mark_line(
    interpolate='basis',
    strokeWidth=2.5,
    opacity=0.25
).encode(
    x=alt.X('Date:T', title=""),
    y=alt.Y('Counts:Q', scale=alt.Scale(domain=olou_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).add_params(
    # param_width,
    select
)

# Olou cosmic ray counts rolling average.
moving_avg_olou = alt.Chart(dataset[np.isnan(dataset['Counts']) == False]).transform_window(
    rolling_mean='mean(Counts)',
    frame=[-15, 15],
).mark_line(
    color='red',
    strokeWidth=2,
    interpolate='basis'
).encode(
    x=alt.X('Date:T'),
    y=alt.Y('rolling_mean:Q', scale=alt.Scale(domain=olou_domain))
)

# Get nearest point y-value from selection.
nearest = alt.selection_point(
    nearest=True,
    on='mouseover',
    fields=['Date'],
    empty=False
)

# Monsoon selector.
selectors_monsoon = alt.Chart(dataset[dataset['dataset'] == 'monsoon']).mark_point().encode(
    x=alt.X('Date:T'),
    y=alt.Y('Precip:Q', scale=alt.Scale(domain=monsoon_domain)),
    opacity=alt.value(0),
).add_params(
    nearest
)

# Olou selector.
selectors_olou = alt.Chart(dataset[dataset['dataset'] == 'olou']).mark_point().encode(
    x=alt.X('Date:T'),
    y=alt.Y('Counts:Q', scale=alt.Scale(domain=olou_domain)),
    opacity=alt.value(0),
).add_params(
    nearest
)

# Monsoon highlighted points.
points_monsoon = alt.Chart(dataset[dataset['dataset'] == 'monsoon']).transform_filter(
    nearest
).mark_point(
    size=90, strokeWidth=1
).encode(
    x=alt.X('Date:T'),
    y=alt.Y('Precip:Q', scale=alt.Scale(domain=monsoon_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
)

# Olou highlighted points.
points_olou = alt.Chart(dataset[dataset['dataset'] == 'olou']).transform_filter(
    nearest
).mark_point(
    size=90, strokeWidth=1
).encode(
    x=alt.X('Date:T'),
    y=alt.Y('Counts:Q', scale=alt.Scale(domain=olou_domain)),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
)

# Gray draggable bar.
rules = alt.Chart(dataset).mark_rule(
    color='lightgray',
    opacity=0.4
).encode(
    x=alt.X('Date:T'),
    strokeWidth=alt.StrokeWidthValue(4)
).transform_filter(
    nearest
)

# Monsoon label text.
text_monsoon = alt.Chart(dataset[dataset['dataset'] == 'monsoon']).mark_text(
    align='left',
    dx=7,
    fontSize=14
).encode(
    x=alt.X('Date:T'),
    y=alt.X('Precip:Q', scale=alt.Scale(domain=monsoon_domain)),
    text=alt.Text('Precip', format='.2f'),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).transform_filter(
    nearest
)

# Olou label text.
text_olou = alt.Chart(dataset[dataset['dataset'] == 'olou']).mark_text(
    align='left',
    dx=7,
    fontSize=14
).encode(
    x=alt.X('Date:T'),
    y=alt.Y('Counts:Q', scale=alt.Scale(domain=olou_domain)),
    text=alt.Text('Counts', format='.2f'),
    color=alt.Color("dataset", legend=alt.Legend(symbolType='circle'))
).transform_filter(
    nearest
)

### All-In-One

In [499]:
# alt.layer(
#     line_chart_monsoon, moving_avg_monsoon, line_chart_olou, moving_avg_olou, points_monsoon, points_olou,
#     selectors_monsoon, selectors_olou, rules, text_monsoon, text_olou
# ).configure_axis(
#     # disable=True,
# ).properties(
#     width=1000, height=400
# ).resolve_scale(x='shared', y='shared')

### Stacked

In [497]:
# chart_olou = alt.layer(
#     line_chart_olou, moving_avg_olou, points_olou, selectors_olou, rules, text_olou
# ).properties(
#     width=1000, height=100
# )

# chart_monsoon = alt.layer(
#     line_chart_monsoon, moving_avg_monsoon, points_monsoon, selectors_monsoon, rules, text_monsoon
# ).properties(
#     width=1000, height=100
# )

# combined_chart = alt.vconcat(chart_olou, chart_monsoon)
# combined_chart

In [500]:
from IPython.display import display
import ipywidgets as widgets

# Init outputs.
output = widgets.Output()

# Update chart sizes.
def update_chart_width_height(width, height):
    updated_chart_olou = chart_olou.properties(width=width, height=height)
    updated_chart_monsoon = chart_monsoon.properties(width=width, height=height)
    updated_combined_chart = alt.vconcat(updated_chart_olou, updated_chart_monsoon)
    with output:
        output.clear_output(wait=True)
        display(updated_combined_chart)

# Create widgets.
width_slider = widgets.IntSlider(value=1000, min=400, max=1600, step=10, description='Width:')
height_slider = widgets.IntSlider(value=100, min=100, max=1600, step=10, description='Height:')

# Update charts from widgets.
def update_chart(width, height):
    update_chart_width_height(width, height)

# Combine chart and widgets.
widgets.interactive(update_chart, width=width_slider, height=height_slider)
display(output)
display(width_slider, height_slider)

Output()

IntSlider(value=1000, description='Width:', max=1600, min=400, step=10)

IntSlider(value=100, description='Height:', max=1600, min=100, step=10)